In [1]:
import re
import sys
import json


import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
from response.feature import Feature

In [ ]:
F_path = "../X_y_data/response2/"
F_name = "Classify_F3.pickle"
featureM = DataManager(F_path)
F = featureM.load_data(F_name)

In [ ]:
model_path = "../models/response2/"
model_name = "Classify_M3.pickle"
modelM = DataManager(model_path)
lr = modelM.load_data(model_name)

In [4]:
path = "../../corpus/hand_labeled/"
# path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [8]:
def make_Xy(convs, N=4):
    errors = ["Ignore question", 'Ignore offer', 'Ignore proposal', "Ignore greeting"]
    # errors = errors[:1]
    
    X = []
    y = []
    for conv in convs:

        dialogue = [""]*N
        for i, ut in enumerate( conv ) :
            # ユーザ発話駆動
            dialogue.append(clean_text( ut.utt) )
            if ut.is_exist_error():
                X.append( dialogue[-N:] )
                    # X.append(dialogue[-N:])
                if ut.is_error_included(errors) :
                    y.append(1)
                else:
                    y.append(0)
        
    return X, y

In [9]:
N = 2
X_str, y = make_Xy(convs, N=N)

In [13]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import models
bert_path = "../../corpus/pretrained/sbert_ignore"
sbert = SentenceTransformer(bert_path)

[11034] 2022-01-20 01:54:35,861 Info sentence_transformers.SentenceTransformer :Load pretrained SentenceTransformer: ../../corpus/pretrained/sbert_ignore
[11034] 2022-01-20 01:54:37,045 Info sentence_transformers.SentenceTransformer :Use pytorch device: cuda


In [14]:
X_all_str = sum(X_str, [])
x_length = len(X_all_str)//N
X_vec = sbert.encode(X_all_str).reshape(x_length, N, -1)

Batches:   0%|          | 0/85 [00:00<?, ?it/s]

In [15]:
X_vec.shape

(1349, 2, 768)

In [17]:
def vec2feature(vector):
    diff = np.abs( vector[0] - vector[1] )
    return np.concatenate([vector.flatten(), diff])

In [18]:
X = np.array([ vec2feature(vec) for vec in X_vec ])

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5, stratify=y)

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import loss
import torch.optim as optim

In [21]:
class Datasets(torch.utils.data.Dataset):
    def __init__(self, X_data, y_data):
        # self.transform = transform

        self.X_data = X_data
        self.y_data = y_data

        self.datanum = len(X_data)

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        out_X = self.X_data[idx]
        out_y = self.y_data[idx]

        return out_X, out_y

In [22]:
class ResponseModel(nn.Module):
    def __init__(self, embedding_dim, tagset_size):
        # 親クラスのコンストラクタ。決まり文句
        super(ResponseModel, self).__init__()    
        self.embedding_dim = embedding_dim
        self.hid1= embedding_dim*2
        self.hid2 = embedding_dim//2
        self.fc1 = nn.Linear(self.embedding_dim, self.hid1)
        self.fc2 = nn.Linear(self.hid1, self.hid2)
        self.hidden2tag = nn.Linear(self.hid2, tagset_size)
    
    def forward(self, x):
        y = F.relu(self.fc1(x))
        y = F.relu(self.fc2(y))
        y = self.hidden2tag( y )
        y = F.log_softmax(y, dim=1)
        return y

In [23]:
BATCH_SIZE = 32
epoch_ = 300
trainset = Datasets(X_train, y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 2)

In [24]:
EMBEDDING_DIM = 768*3
HIDDEN_DIM = EMBEDDING_DIM*2
OUTPUT_DIM = 2
# seq_len = length
print(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

2304 4608 2


In [25]:
model = ResponseModel(EMBEDDING_DIM, OUTPUT_DIM)
if torch.cuda.is_available():
   model.cuda()
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00005)

In [26]:
losses = []
for epoch in tqdm( range(epoch_)  ):  # again, normally you would NOT do 300 epochs, it is toy data
    all_loss = 0
    for data in trainloader:
        # X_t_tensor = torch.tensor(data[0], device='cuda:0', dtype=torch.int16)
        X_t_tensor = data[0].cuda()
        y_t_tensor = data[1].to(torch.long).cuda()
        optimizer.zero_grad()
        model.zero_grad()
        # print(X_t_tensor.shape , y_t_tensor.view(-1,1).shape)

        score_ = model(X_t_tensor)
        # print(X_t_tensor.shape, score.view(-1,5).shape, y_t_tensor.view(-1,1).shape)
        loss_ = loss_function(score_,  y_t_tensor)
        loss_.backward()
        all_loss += loss_.item()
        optimizer.step()
        del score_
        del loss_
    losses.append(all_loss)
    if (epoch+1) % 20 == 0:
        print("epoch", epoch+1, "\t" , "loss", all_loss)
print("done")

  7%|▋         | 20/300 [00:09<02:07,  2.20it/s]

epoch 20 	 loss 0.04338508303044364


 13%|█▎        | 40/300 [00:18<01:55,  2.26it/s]

epoch 40 	 loss 0.0053000973057351075


 20%|██        | 60/300 [00:26<01:46,  2.26it/s]

...

epoch 180 	 loss 4.7757960828676005e-05


 67%|██████▋   | 200/300 [01:28<00:43,  2.28it/s]

epoch 200 	 loss 3.159034719146803e-05


 70%|███████   | 211/300 [01:34<00:39,  2.24it/s]


KeyboardInterrupt: 

In [27]:
with torch.no_grad():
    X_tensor = torch.tensor(X_test, device='cuda:0').float()
            # 推論
    y_pred= np.array(model(X_tensor).cpu()).argmax(axis=1)

In [29]:
score(y_test, y_pred)

confusion matrix = 
 [[286  26]
 [ 11  82]]
accuracy =  0.908641975308642
precision =  0.7592592592592593
recall =  0.8817204301075269
f1 score =  0.8159203980099504


In [30]:
smodel_path = "../models/response2/"
smodel_name = "sbert_ffn.pickle"
smodelM = DataManager(smodel_path)
smodelM.save_data(smodel_name, model)

success save : ../models/response2/sbert_ffn.pickle
